In [ ]:
# Import essential libraries
import pandas as pd

%matplotlib inline

# Import other libraries
import os
import sys

# Import our own libraries
sys.path.append("../lib/")
import eyegaze as eg

In [ ]:
# 全参加者・全フェーズの定義
participants = {
    "A": ["P001", "P002", "P005", "P006", "P008", "P009", "P010", "P011", "P016", "P017"],
    "B": ["P003", "P004", "P007", "P012", "P013", "P014", "P015", "P018", "P019", "P020"],
}
phases = ["pre", "post", "training1", "training2", "training3"]

# 補正パラメータの読み込み（全参加者分を一括で読み込む）
corrections_path = "../../data/working/corrections/all_segment_corrections.csv"
all_corrections = pd.read_csv(corrections_path)
print(f"補正パラメータ: {len(all_corrections)}行")

# 処理対象の確認
total = sum(len(pids) for pids in participants.values()) * len(phases)
print(f"処理対象: {total}件（{sum(len(pids) for pids in participants.values())}名 × {len(phases)}フェーズ）")

In [ ]:
# 全参加者・全フェーズの一括処理（並列版）
from concurrent.futures import ProcessPoolExecutor, as_completed

# タスクリストの構築
corrections_path = "../../data/working/corrections/all_segment_corrections.csv"
tasks = []
for group_letter, participant_ids in participants.items():
    for participant_id in participant_ids:
        for phase in phases:
            tasks.append({
                'group_letter': group_letter,
                'participant_id': participant_id,
                'phase': phase,
                'corrections_path': os.path.abspath(corrections_path),
                'input_root': os.path.abspath('../../data/input'),
                'output_root': os.path.abspath('../../data/output'),
                'working_root': os.path.abspath('../../data/working'),
            })

total = len(tasks)
n_workers = min(os.cpu_count(), total)
print(f"Total: {total} tasks, Workers: {n_workers}")

# 並列実行
completed_count = 0
error_count = 0
errors = []

with ProcessPoolExecutor(max_workers=n_workers) as executor:
    futures = {executor.submit(eg._processOnePhaseWorker, t): t for t in tasks}
    for future in as_completed(futures):
        result = future.result()
        if result['success']:
            completed_count += 1
            print(f"  [{completed_count + error_count}/{total}] OK: {result['label']} ({result['n_segments']} segments)")
        else:
            error_count += 1
            errors.append((result['label'], result['error']))
            print(f"  [{completed_count + error_count}/{total}] ERROR: {result['label']} - {result['error'].splitlines()[0]}")

# --- 結果サマリー ---
print(f"\n{'='*60}")
print(f"完了: {completed_count}/{total}件 (エラー: {error_count}件)")
if errors:
    for label, msg in errors:
        print(f"  {label}: {msg}")

In [ ]:
# === AOI一致率の検証（並列版）: gaze-correctionの結果と一致するか確認 ===
from concurrent.futures import ProcessPoolExecutor, as_completed
import matplotlib.pyplot as plt

TOLERANCE = 5.0
input_root = os.path.abspath('../../data/input')
corrections_root = os.path.abspath('../../data/working/corrections')

# タスクリストの構築
verify_tasks = []
for group_letter, participant_ids in participants.items():
    for participant_id in participant_ids:
        for phase in phases:
            verify_tasks.append({
                'group_letter': group_letter,
                'participant_id': participant_id,
                'phase': phase,
                'input_root': input_root,
                'corrections_root': corrections_root,
                'tolerance': TOLERANCE,
            })

total = len(verify_tasks)
n_workers = min(os.cpu_count(), total)
print(f"Total: {total} tasks, Workers: {n_workers}")

all_comparisons = []
completed_count = 0
error_count = 0

with ProcessPoolExecutor(max_workers=n_workers) as executor:
    futures = {executor.submit(eg._verifyAOIRateWorker, t): t for t in verify_tasks}
    for future in as_completed(futures):
        result = future.result()
        if result['success']:
            completed_count += 1
            all_comparisons.extend(result['comparisons'])
            print(f"  [{completed_count + error_count}/{total}] OK: {result['label']} ({len(result['comparisons'])} segments)")
        else:
            error_count += 1
            print(f"  [{completed_count + error_count}/{total}] ERROR: {result['label']} - {result['error'].splitlines()[0]}")

comparison_df = pd.DataFrame(all_comparisons)
comparison_df['rate_diff'] = comparison_df['eb_original_rate'] - comparison_df['vr_original_rate']
comparison_df['fixation_diff'] = comparison_df['eb_fixations'] - comparison_df['vr_fixations']

print(f"\n比較対象セグメント数: {len(comparison_df)}")
print(f"\n=== 補正前AOI一致率の比較 ===")
print(f"  差の平均:   {comparison_df['rate_diff'].mean():.6f}")
print(f"  差の最大:   {comparison_df['rate_diff'].abs().max():.6f}")
print(f"  完全一致:   {(comparison_df['rate_diff'] == 0).sum()}/{len(comparison_df)}")
print(f"  近似一致 (|diff| < 0.001): {(comparison_df['rate_diff'].abs() < 0.001).sum()}/{len(comparison_df)}")
print(f"\n=== fixation数の比較 ===")
print(f"  差の平均:   {comparison_df['fixation_diff'].mean():.1f}")
print(f"  完全一致:   {(comparison_df['fixation_diff'] == 0).sum()}/{len(comparison_df)}")

In [ ]:
if len(comparison_df) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # 散布図: event-based vs validation
    axes[0].scatter(comparison_df['vr_original_rate'], comparison_df['eb_original_rate'],
                    alpha=0.4, s=15)
    axes[0].plot([0, 1], [0, 1], 'r--', label='y = x')
    axes[0].set_xlabel('gaze-correction original_rate')
    axes[0].set_ylabel('event-based original_rate')
    axes[0].set_title('Original AOI Rate Comparison')
    axes[0].legend()
    axes[0].set_xlim(0, 1)
    axes[0].set_ylim(0, 1)

    # 差分のヒストグラム
    axes[1].hist(comparison_df['rate_diff'], bins=50, edgecolor='black', alpha=0.7)
    axes[1].axvline(x=0, color='red', linestyle='--')
    axes[1].set_xlabel('Rate Difference (event-based − gaze-correction)')
    axes[1].set_ylabel('Count')
    axes[1].set_title('Distribution of Rate Differences')

    # fixation数の差分
    axes[2].hist(comparison_df['fixation_diff'], bins=30, edgecolor='black', alpha=0.7)
    axes[2].axvline(x=0, color='red', linestyle='--')
    axes[2].set_xlabel('Fixation Count Difference')
    axes[2].set_ylabel('Count')
    axes[2].set_title('Distribution of Fixation Count Differences')

    plt.tight_layout()
    plt.show()

In [ ]:
per_participant = comparison_df.groupby(['group', 'participant']).agg(
    n_segments=('segment_id', 'count'),
    mean_diff=('rate_diff', 'mean'),
    max_abs_diff=('rate_diff', lambda x: x.abs().max()),
    fixation_exact_match=('fixation_diff', lambda x: (x == 0).sum()),
).round(6)
print("=== 参加者別サマリー ===")
print(per_participant.to_string())